Atividade #11: Indo Além (Hugging Face)

Integrantes: Jorge Mutran, kaick Nishiya, Lucas koiti, Lucas Prado, Lucas Quinto e Matheus Pestilli

Passo 1: Tópico Escolhido
Tópico: 1 RAG (Retrieved Augmented Generation)

Passo 2: Construção da Aplicação (Código Python)
Vamos criar um "Chatbot Especialista em Filmes". Ele terá uma pequena base de conhecimento (que o modelo original não conhece necessariamente em detalhes) e usará RAG para responder.

Instalação das bibliotecas

In [7]:
# Instalando as bibliotecas necessárias do ecossistema Hugging Face e vetores
!pip install -q transformers sentence-transformers faiss-cpu

 O Código da Aplicação RAG

In [8]:
# --- BLOCO DE LIMPEZA (Adicionado para remover avisos) ---
import warnings
import logging
import os

# Ignorar warnings gerais do Python
warnings.filterwarnings("ignore")

# Desligar logs informativos das bibliotecas
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" # Para TensorFlow/Keras
logging.getLogger("sentence_transformers").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
# -------------------------------------------------------

import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

# --- 1. Base de Conhecimento ---
documents = [
    "O filme 'A Jornada do Dev' foi lançado em 2024 e conta a história de um programador que descobriu um bug na Matrix.",
    "A linguagem de programação Python foi criada por Guido van Rossum e lançada em 1991.",
    "Para instalar bibliotecas no Python, utilizamos o gerenciador de pacotes chamado pip.",
    "O framework Hugging Face facilita o uso de modelos de Deep Learning para processamento de linguagem natural."
]

# --- 2. Indexação ---
print("Iniciando aplicação RAG...") # Mudei o texto para ficar mais bonito
encoder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = encoder.encode(documents)

d = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(doc_embeddings)

# --- 3. Função de Busca ---
def search(query, k=1):
    query_vector = encoder.encode([query])
    distances, indices = index.search(query_vector, k)
    return documents[indices[0][0]]

# --- 4. Geração da Resposta ---
# O parâmetro model_kwargs ajuda a suprimir alguns avisos internos também
generator = pipeline('text2text-generation', model='google/flan-t5-base')

def rag_chatbot(user_query):
    print(f"\nPergunta: {user_query}") # O \n pula uma linha para ficar organizado

    context = search(user_query)
    print(f"Contexto Encontrado: {context}")

    prompt = f"Context: {context}\n\nQuestion: {user_query}\n\nAnswer based on the context:"

    # Usando max_new_tokens para evitar o warning de tamanho
    response = generator(prompt, max_new_tokens=100)[0]['generated_text']
    return response

# --- 5. Testando ---
print("-" * 50)
resposta = rag_chatbot("Quando foi lançado o filme A Jornada do Dev?")
print(f"Resposta da IA: {resposta}")

print("-" * 50)
resposta_tec = rag_chatbot("Quem criou o Python?")
print(f"Resposta da IA: {resposta_tec}")

Iniciando aplicação RAG...
--------------------------------------------------

Pergunta: Quando foi lançado o filme A Jornada do Dev?
Contexto Encontrado: O filme 'A Jornada do Dev' foi lançado em 2024 e conta a história de um programador que descobriu um bug na Matrix.
Resposta da IA: 2024
--------------------------------------------------

Pergunta: Quem criou o Python?
Contexto Encontrado: A linguagem de programação Python foi criada por Guido van Rossum e lançada em 1991.
Resposta da IA: Guido van Rossum
